In [12]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
import copy
%matplotlib inline
import skimage.segmentation
from sklearn.linear_model import LinearRegression
from skimage.color import gray2rgb
import os
from PIL import Image
import json
import pickle


In [13]:
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
from sklearn.svm import SVC

In [14]:
import shap
import cv2
import matplotlib
from PIL import Image

import shap
import matplotlib
import joblib

In [15]:
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

In [16]:

# read by default 1st sheet of an excel file
# df = pd.read_excel('xai_thyroid/Data/labels.xlsx', header=0)
# df.to_csv('xai_thyroid/Data/labels.csv', index=False)
dataframe = pd.read_csv("../xai_thyroid/Data/labels.csv", encoding='utf-8')
dataframe = dataframe[:480]
# print(dataframe)

dataset = []
for i in range(len(dataframe)):
    if dataframe["Cla1"][i]==0:
        continue
    row_data = []
    
    for row in dataframe:
        row_data.append(dataframe[row][i])
    dataset.append(row_data)

dataset

[['2_1', 1, '2'],
 ['3_1', 2, '4a'],
 ['4_1', 2, '4a'],
 ['5_1', 2, '5'],
 ['6_1', 2, '4b'],
 ['7_1', 2, '4a'],
 ['8_1', 2, '4c'],
 ['9_1', 2, '4b'],
 ['10_1', 2, '4b'],
 ['11_1', 2, '5'],
 ['12_1', 2, '4a'],
 ['13_1', 2, '4a'],
 ['14_1', 1, '2'],
 ['15_1', 2, '4c'],
 ['16_1', 1, '3'],
 ['17_1', 1, '2'],
 ['18_1', 2, '4a'],
 ['19_1', 2, '5'],
 ['20_1', 2, '4b'],
 ['21_1', 2, '4a'],
 ['22_1', 1, '2'],
 ['23_1', 2, '4a'],
 ['24_1', 2, '4b'],
 ['25_1', 1, '3'],
 ['26_1', 2, '4a'],
 ['27_1', 2, '4b'],
 ['28_1', 2, '4b'],
 ['29_1', 2, '5'],
 ['30_1', 2, '4c'],
 ['31_1', 1, '3'],
 ['32_1', 2, '4a'],
 ['33_1', 2, '4a'],
 ['34_1', 2, '4c'],
 ['35_1', 2, '4b'],
 ['36_1', 2, '5'],
 ['38_1', 2, '4b'],
 ['39_1', 2, '4a'],
 ['40_1', 2, '4a'],
 ['41_1', 2, '4b'],
 ['42_1', 2, '4b'],
 ['43_1', 2, '4c'],
 ['44_1', 2, '4a'],
 ['45_1', 2, '4b'],
 ['46_1', 2, '5'],
 ['47_1', 2, '4a'],
 ['48_1', 2, '4b'],
 ['49_1', 2, '4a'],
 ['50_1', 2, '4a'],
 ['51_1', 1, '2'],
 ['52_1', 2, '4a'],
 ['53_1', 2, '4c'],
 [

In [17]:
def print_annotated_image_by_doctor(xml_url,image_path):
    lst=list()
    tree = ET.parse(xml_url)
    root = tree.getroot()
    for child in root:
        if child.tag=='mark':
            for child_child in child:
                if child_child.tag=='svg':
                    lst=json.loads(child_child.text)
    points=lst[0]["points"]
    n=len(lst[0]["points"])
    coordinates=np.zeros((n,2))
    for i in range(n):
        coordinates[i][0]=points[i]['x']
        coordinates[i][1]=points[i]['y']
    image = skimage.io.imread(image_path)
#     print(type(image))
    plt.imshow(image)
    plt.plot("og", markersize=10)  # og:shorthand for green circle
    plt.scatter(coordinates[:, 0], coordinates[:, 1], marker="o", color="yellow", s=20)
    plt.show()
#     print(image.shape)
    return image

In [18]:
import time
import shutil

In [2]:
def print_results(results_folder):
    it=0
    count=0

    source_folder = 'Not Correctly Explained'

    # Iterate through files in the source folder
    for filename in os.listdir(source_folder):
        filename = filename[:-4]
        if filename[-1]=='.':
            filename = filename[:-1]

        foldername = 0
        for filename1 in os.listdir(results_folder + "1/"):
            filename1 = filename1[:-4]
            if filename==filename1:
                foldername=1
                break

        for filename1 in os.listdir(results_folder + "2/"):
            filename1 = filename1[:-4]
            if filename==filename1:
                foldername=2
                break

        row = [filename, foldername]
        if row[0]=="54_1" or row[0]=="127_1" or row[0]=="142_1" or row[0]=="142_2" or row[0]=="166_1" or row[0]=="176_1":
            continue
        if row[0]=="197_1" or row[0]=="203_1" or row[0]=="205_1":
            continue

        count=count+1
    #     if count>2:
    #         break

        print(count)
        print(row[0], " ",)
        print(row[1]," ")
        if row[1]==1:
            print("Benign\nIn prediction, Green -> Benign and Red -> Malignant\nIn Heat map, Blue -> Benign and Brown -> Malignant")
        if row[1]==2:
            print("Malignant\nIn prediction, Green -> Malignant and Red -> Benign\nIn Heat map, Blue -> Malignant and Brown -> Benign")

        imgno = str(row[0])
        imgno = imgno.split("_")
        xml_url = "../xai_thyroid/Data/archive/" + imgno[0] + ".xml"
        image_path="../xai_thyroid/Data/archive/" + str(row[0]) + ".jpg"
        img = print_annotated_image_by_doctor(xml_url,image_path)

        ex_filename = results_folder + str(row[1]) + "/" + str(row[0]) + '.bz2'
        shap_values1 = joblib.load(filename=ex_filename)
        shap.image_plot(shap_values1/255)

In [3]:
print_results("../1. SHAP Code/Results/HOT-SVM Model Results/")

NameError: name 'os' is not defined